# ML-box Avocado

#### Goal :

- Create a ML model using Auto-sklearn for the Avocado dataset
- Get RMSE over the predictions of these model

#### Imports

In [1]:
import numpy as np
import pandas as pd
import sklearn.metrics
from math import sqrt
from mlbox.preprocessing import Reader
from mlbox.preprocessing import Drift_thresholder
from mlbox.optimisation import make_scorer
from mlbox.optimisation import Optimiser
from mlbox.prediction import Predictor


reading csv : train.csv ...
cleaning data ...
CPU time: 0.6825635433197021 seconds

reading csv : test.csv ...
cleaning data ...
CPU time: 0.45384645462036133 seconds

> Number of common features : 80

gathering and crunching for train and test datasets ...
reindexing for train and test datasets ...
dropping training duplicates ...
dropping constant variables on training set ...

> Number of categorical features: 43
> Number of numerical features: 37
> Number of training samples : 1460
> Number of test samples : 1459

> Top sparse features (% missing values on train set):
PoolQC         99.5
MiscFeature    96.3
Alley          93.8
Fence          80.8
FireplaceQu    47.3
dtype: float64

> Task : regression
count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

computing drifts ...
CPU time: 3.251035213470459 seconds

> Top 10 

/opt/conda/lib/python3.7/site-packages/mlbox/optimisation/optimiser.py:74: UserWarning: Optimiser will save all your fitted models into directory 'save/joblib'. Please clear it regularly.
  +str(self.to_path)+"/joblib'. Please clear it regularly.")


Paths to the train set and the test set.

In [ ]:
paths = ["../../Data/California_house/train.csv", "../../Data/California_house/test.csv"]

Name of the feature to predict.
This columns should only be present in the train set.

In [ ]:
target_name = "SalePrice"

Reading and cleaning all files.
Declare a reader for csv files.

In [ ]:
rd = Reader(sep=',')

Return a dictionnary containing three entries:
- dict["train"] contains training samples withtout target columns
- dict["test"] contains testing elements withtout target columns
- dict["target"] contains target columns for training samples.

In [ ]:
data = rd.train_test_split(paths, target_name)

Removing the drifting variables

In [ ]:
dft = Drift_thresholder()
data = dft.fit_transform(data)

Tuning

In [ ]:
mape = make_scorer(lambda y_true,
                   y_pred: 100*np.sum(
                                      np.abs(y_true-y_pred)/y_true
                                      )/len(y_true),
                   greater_is_better=False,
                   needs_proba=False)

Declare an optimiser

In [ ]:
opt = Optimiser(scoring="mean_squared_error", n_folds=3)

Space of hyperparameters

In [ ]:
space = {
        'ne__numerical_strategy': {"search": "choice",
                                   "space": [0]},
        'ce__strategy': {"search": "choice",
                         "space": ["label_encoding",
                                   "random_projection",
                                   "entity_embedding"]},
        'fs__threshold': {"search": "uniform",
                          "space": [0.01, 0.3]},
        'est__max_depth': {"search": "choice",
                           "space": [3, 4, 5, 6, 7]}

        }

Optimises hyper-parameters of the whole Pipeline

In [2]:
best = opt.optimise(space,data,40)
print("Final results : " ,opt.evaluate(best, data))

##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 0, 'categorical_strategy': '<NULL>'}
>>> CA ENCODER :{'strategy': 'entity_embedding'}      
  0%|          | 0/40 [00:00<?, ?trial/s, best loss=?]

/opt/conda/lib/python3.7/site-packages/mlbox/optimisation/optimiser.py:296: UserWarning: Unknown or invalid scoring metric. neg_mean_squared_error is used instead.
  warnings.warn("Unknown or invalid scoring metric. "



>>> FEATURE SELECTOR :{'strategy': 'l1', 'threshold': 0.26606117093345866}
>>> ESTIMATOR :{'strategy': 'LightGBM', 'max_depth': 4, 'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.8, 'importance_type': 'split', 'learning_rate': 0.05, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 500, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': True, 'subsample': 0.9, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'nthread': -1, 'seed': 0}
  0%|          | 0/40 [00:00<?, ?trial/s, best loss=?]

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 466381221651.57007, tolerance: 577337156.9466883
  positive)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 322247582041.4366, tolerance: 630960719.5801489
  positive)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 437203963679.869, tolerance: 633252671.051735
  positive)



MEAN SCORE : neg_mean_squared_error = -900921299.7172343
VARIANCE : 351178789.0767834 (fold 1 = -721565509.653147, fold 2 = -1391677040.4027362, fold 3 = -589521349.09582)
CPU time: 7.061063766479492 seconds                   
##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 0, 'categorical_strategy': '<NULL>'}    
>>> CA ENCODER :{'strategy': 'entity_embedding'}                               
>>> FEATURE SELECTOR :{'strategy': 'l1', 'threshold': 0.1480372523094022}      
>>> ESTIMATOR :{'strategy': 'LightGBM', 'max_depth': 6, 'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.8, 'importance_type': 'split', 'learning_rate': 0.05, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 500, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': Tr

...

MEAN SCORE : neg_mean_squared_error = -874419386.3908612                        
VARIANCE : 330204400.4533249 (fold 1 = -697455324.019692, fold 2 = -1337154286.0890722, fold 3 = -588648549.0638192)
CPU time: 3.8954994678497314 seconds                                            
##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 0, 'categorical_strategy': '<NULL>'}     
>>> CA ENCODER :{'strategy': 'label_encoding'}                                  
>>> FEATURE SELECTOR :{'strategy': 'l1', 'threshold': 0.13537083446910095}      
>>> ESTIMATOR :{'strategy': 'LightGBM', 'max_depth': 4, 'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.8, 'importance_type': 'split', 'learning_rate': 0.05, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 500, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': 

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 382002478923.82025, tolerance: 577337156.9466883
  positive)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 226417788333.6526, tolerance: 630960719.5801489
  positive)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 326095800875.1044, tolerance: 633252671.051735
  positive)



MEAN SCORE : neg_mean_squared_error = -941211715.9882464                        
VARIANCE : 358309396.5112269 (fold 1 = -733572359.6411499, fold 2 = -1445334852.4458594, fold 3 = -644727935.8777298)
CPU time: 3.1373825073242188 seconds                                            
100%|██████████| 40/40 [02:54<00:00,  4.36s/trial, best loss: 864638297.3940634]


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ BEST HYPER-PARAMETERS ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

{'ce__strategy': 'random_projection', 'est__max_depth': 3, 'fs__threshold': 0.10271650679151925, 'ne__numerical_strategy': 0}

##################################################### testing hyper-parameters... #############

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 353915657654.1744, tolerance: 577337156.9466883
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 212966893949.87384, tolerance: 630960719.5801489
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 293134499318.21173, tolerance: 633252671.051735
  positive)



MEAN SCORE : neg_mean_squared_error = -864638297.3940634
VARIANCE : 335690464.5193856 (fold 1 = -668112057.0292984, fold 2 = -1337154286.0890722, fold 3 = -588648549.0638192)
CPU time: 3.86971378326416 seconds

Final results :  -864638297.3940634


Make a prediction

In [ ]:
from mlbox.prediction import *
pred=Predictor()
pred.fit_predict(best,data)

Getting the predictions and targets

In [4]:
y_pred = pd.read_csv("save/SalePrice_predictions.csv")
predictions = y_pred.AveragePrice_predicted
y_test = pd.read_csv("../../Data/California_house/processed/y_test.csv")

,Unnamed: 0,SalePrice_predicted
count,1459.000000,1459.000000
mean,729.000000,179022.867991
std,421.321334,77879.341567
min,0.000000,43400.770114
25%,364.500000,128148.813415
50%,729.000000,158642.079861
75%,1093.500000,211934.415030
max,1458.000000,519868.288239


Calculating RMSE

In [5]:
from sklearn.metrics import mean_squared_error
from math import sqrt
print("rmse score:", sqrt(mean_squared_error(y_test, predictions)))

In [11]:
from sklearn.metrics import mean_squared_error
from math import sqrt
print("rmse score:", sqrt(mean_squared_error(y_test, predictions)))

rmse score: 110545.05648529054


## Residual Plot

In [16]:
import matplotlib.pyplot as plt

plt.scatter(predictions, score, c="grey", label="Testing Data")
plt.legend()
plt.hlines(y=0, xmin=y_test.min(), xmax=y_test.max())
plt.title("Residual Plot")
plt.show()